# High-level Algorithm Design with Qmod
# Workshop - Language Concepts

In this workshop we will learn to use high-level quantum programming language concepts to design quantum algorithms. We will use the Qmod language to model the functionality, and the Classiq platform to synthesize it into gate-level descriptions, visualize the circuits, and execute them. We will focus on high-level quantum types and expressions in different evaluation modes.

Part I is a walk through Qmod's structure and constructs, as well as some of its unique high-level concepts. We will look closely at these constructs using small examples. In Part II we will combine some of these concepts in a complete quantum algorithm.

There are 5 code exercises in this notebook, split into two sections, *A* and *B*. In each exercise follow the instructions - complete the code snippet where indicated by a *TODO* comment, execute the code, and try to understand the results. Solutions are provided at the end of the notebook. Don't continue to the next exercise before you completed the previous one and compared your code and results against the solution.

## Section A *(15 minutes)*

### Warmup: A First Qmod Program

Let's start with a simple example to demonstrate the structure of Qmod code, as well as its synthesis and execution flow. We prepare and sample a **Bell state** - one of the most fundamental quantum states.

Further reading:
[Quantum Functions](https://docs.classiq.io/latest/qmod-reference/language-reference/functions/)

In [9]:
from classiq import *


@qfunc  # This decorator declares a quantum function
def create_bell_state(pair: QArray[QBit, 2]):
    H(pair[0])
    CX(pair[0], pair[1])


@qfunc  # Function 'main' is the entry point of our quantum program
def main(res: Output[QArray[QBit, 2]]):
    allocate(res)
    create_bell_state(res)

*<div class="alert alert-info">Quantum functions in Qmod are defined using a regular Python function, decorated with **qfunc**, and their parameters must be declared with type hints.</div>*

We can now compile with the SDK function `synthesize`. We get back an executable description called *quantum program* which we then execute on any simulation or quantum hardware. To manage the execution flow we use `ExecutionSession`. In our case, a simple sampling (using the default number of shots) of the quantum program will suffice.

In [10]:
qprog = synthesize(main)

show(qprog)  # Visualize the quantum program for analysis

# Execute and print the results:
with ExecutionSession(qprog) as es:
    res = es.sample()
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31oOs8PriN4eciU6OrR5HzP1Lyg


https://platform.classiq.io/circuit/31oOs8PriN4eciU6OrR5HzP1Lyg?login=True&version=0.90.0

,res,count,probability,bitstring
0,"[0, 0]",1040,0.507812,00
1,"[1, 1]",1008,0.492188,11


*<div class="alert alert-primary">You should see in approximately 50% of the samples the bit vector 00 and in 50% the bit vector 11.</div>*

*<div class="alert alert-info">Synthesis is the process of compiling a high-level description to a gate-level description. The reduction is presented graphically. The executable format can be simulated using various simulation engines, or executed on quantum hardware of choice.</div>*

### Exercise 1: GHZ State

Based on the Bell state example above, prepare a **GHZ state** with 3 qubits. The GHZ state creates maximum entanglement between three qubits: (|000⟩ + |111⟩)/√2.

In [11]:
from classiq import *


@qfunc
def create_ghz_state(qubits: QArray[QBit, 3]):
    # TODO: Apply GHZ logic
    pass


@qfunc
def main(res: Output[QArray[QBit, 3]]):
    allocate(res)
    create_ghz_state(res)


# TODO: Synthesize the model, show, execute and print results
# Hint: Follow the same pattern as the demonstration above

*<div class="alert alert-primary">You should see in approximately 50% of the samples the bit vector 000 and in 50% the bit vector 111.</div>*

### Exercise 2: GHZ with Numeric Variables

Define a main function that calls the `create_ghz_state` (as implemented in Exercise 1) with a ***signed*** quantum number and outputs the results.

Synthesize it, analyze the quantum program, execute it and print the results. What do you expect the resulting value of `x` to be? 

Further reading: [Quantum Types](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-types/)

In [12]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 0]]):
    # TODO: Create GHZ state with QNum (gives 0 and -1)
    # Hint: Is it any different from the previous main implementation?
    pass


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">You should see approximately 50% for 0 and 50% for -1.</div>*

*<div class="alert alert-info">In Qmod function arguments are automatically cast between **QNum** and **QArray[QBit]** (and also between other quantum types).
    The value of quantum variables is interpreted based on their type. The state $|111\rangle$ represents 7 for an unsigned integer, and -1 as signed integer (in two's complement encoding).</div>*

## Section B *(30 minutes)*

### Exercise 3: Arithmetic Expressions with Automatic Type Inference

#### Part A: Numeric type inference

Create and execute a quantum program that assigns a quantum arithmetic expression to a numeric variable:
1. Declare quantum numeric variables `a` and `b` as unsigned integers of size of 2.
2. Apply `hadamard_transform` on `a` and `b` (to put them in uniform superposition of all possible states).
3. Assign the value of `3*a + b` to `c`.
4. Synthesize, show, execute and print results.
 Inspect the printouts - what numeric attributes were inferred for variable `c`? Why?

Further reading: [Numeric Assignment](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/numeric-assignment/)


In [13]:
from classiq import *


@qfunc
def main(a: Output[QNum[2]], b: Output[QNum[2]], c: Output[QNum]):
    allocate(a)
    allocate(b)
    # TODO: Put a and b in equal superposition

    # TODO:Assign the value of 3*a + b to c

    # Print out c's inferred size in qubits
    print(f"The size of c is {c.size}")


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">The expression result ranges between 0 and 12. To represent all values variable `c` must be an unsigned integer of size 4 qubits or more.</div>*

*<div class="alert alert-info">In Qmod the size of numeric variables may be left unspecified. It is then automatically inferred to tightly fit all possible values of the expressions.</div>*

#### Part B: Numeric type inference with fixed-point fractions

Repeat *Part A*, only this time declaring `a` with 1 fraction digit and `b` with 2 fraction digits. How did the numeric attributes of `c` change? What are the corresponding sampled values for `c` in the result?


In [14]:
from classiq import *


@qfunc
def main(
    a: Output[QNum[2, UNSIGNED, 1]], b: Output[QNum[2, UNSIGNED, 2]], c: Output[QNum]
):
    allocate(a)
    allocate(b)
    # TODO: Put a and b in equal superposition

    # TODO: Assign the value of 3*a + b to c
    # Hint: Is it any different from the previous main implementation?

    # Print out the numeric attributes of the inferred type
    print("Numeric attributes of c:")
    print(
        f"size={c.size}, is_signed={c.is_signed}, fraction_digits={c.fraction_digits}"
    )


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">Variable `c` should be of size 5, unsigend, and with 2 fraction digits. This is the minimal type that covers the expression's domain.</div>*

*<div class="alert alert-info">In Qmod, numeric variables can represent arbitrary fixed-point values. Arithemetic expression and type inference also accommodate for different decimal point locations.</div>*

### Exercise 4: Conditional Operations

Define a main function that initializes a quantum variable `x` (a 3-qubit signed number with 2 binary fraction digits) in an equal superposition of all states, then conditionally flips a single-qubit variable named `flag` when the value of x is less than 0.5. Inspect the execution results - how is `flag` entangled with `x`?

Further reading: [Control Statement](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/control/)

In [15]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 2]], flag: Output[QBit]):
    allocate(x)
    hadamard_transform(x)

    allocate(flag)
    # TODO : Flip the state of flag if x < 0.5


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">You should see that `x` is evenly distributed across the 8 values, and `flag` is flipped in 6 out of the 8 cases.</div>*

*<div class="alert alert-info">The **control** statement in Qmod is similar to classical **if** statement, where a statement block is applied conditionally, depending on a Boolean expression, and optionally an else-block is applied otherwise. The difference is that the operations are applied in superposition, corresponding to the condition.</div>*


### Exercise 5: Computing x² Using Phase Operations

#### Part A: Phase Encoding x²

Compute x² in the phase of its state. To actually view the phases of the states simulate the program using state-vector simulation.
Expand the quantum program visualization down to the gate-level implementation. How is the 'phase' statement synthesized?
Inspect the resulting phases of the different states. Do they match the phase expression?

Further reading: [Phase Statement](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/phase/)

In [16]:
import numpy as np

from classiq import *
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences


@qfunc
def main(x: Output[QNum[3]]):
    allocate(x)
    # TODO: put x in uniform superposition

    # TODO: Encode x² into the phase


# Synthesize the model and show
qprog = synthesize(main)
show(qprog)

# Specify execution preferences for state vector simulation
preferences = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(backend_name="simulator_statevector"),
)

# Execute and print results:
with ExecutionSession(qprog, preferences) as es:
    res = es.sample()
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31oOskef6bqoQRifgrQYM6PR7rB


https://platform.classiq.io/circuit/31oOskef6bqoQRifgrQYM6PR7rB?login=True&version=0.90.0

,x,amplitude,probability,bitstring
0,0,1.0+0.0j,1.0,000



*<div class="alert alert-secondary">The resulting state phases should show x² rotation of the respective computational-state value, modulo 8 (determined by the domain of variable `x`). The steps are a 1/8 of a full $2\pi$ rotation</div>*

*<div class="alert alert-info">The **phase** statement can operate on an arbitrary polynomials over quantum numeric variables, introducing the corresponding Z rotations on the respective states.</div>*

#### Bonus: Fourier Arithmetic

Create a quantum program that computes res = x² by encoding the result in the phase of a Fourier basis. Then transform the result back to the computational basis. Inspect the execution results to validate the correctness of your algorithm.

In [17]:
import numpy as np

from classiq import *


@qfunc
def main(x: Output[QNum[3]], res: Output[QNum[4]]):
    allocate(x)
    hadamard_transform(x)
    allocate(res)

    # TODO: Use within_apply with QFT and phase operations


# Synthesize the model and show
qprog = synthesize(main)
# show(qprog)

# Synthesize the model, show, execute and print results
with ExecutionSession(qprog) as es:
    res = es.sample()
    display(res.dataframe)

,x,res,count,probability,bitstring
0,7,0,272,0.132812,0000111
1,4,0,271,0.132324,0000100
2,2,0,267,0.130371,0000010
3,6,0,259,0.126465,0000110
4,1,0,251,0.122559,0000001
5,5,0,251,0.122559,0000101
6,0,0,241,0.117676,0000000
7,3,0,236,0.115234,0000011


*<div class="alert alert-info">The **phase** statement can be used to implement modular arithmetic in the Fourier basis.</div>*

## Solutions

### Solution 1: GHZ State


In [18]:
from classiq import *


@qfunc
def create_ghz_state(qubits: QArray[QBit, 3]):
    # Apply GHZ logic
    H(qubits[0])
    CX(qubits[0], qubits[1])
    CX(qubits[1], qubits[2])


@qfunc
def main(res: Output[QArray[QBit, 3]]):
    allocate(res)
    create_ghz_state(res)


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.1 Results:")
    display(res.dataframe)

Task exception was never retrieved
future: <Task finished name='Task-54' coro=<ApiWrapper.call_create_session_job() done, defined at /home/openvscode-server/classiq-venv/lib/python3.12/site-packages/classiq/_internals/api_wrapper.py:227> exception=ReadError('')>
Traceback (most recent call last):
  File "/home/openvscode-server/classiq-venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/home/openvscode-server/classiq-venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/openvscode-server/classiq-venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/home/openvscode-server/classiq-venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_asy

Quantum program link: https://platform.classiq.io/circuit/31oOtYmg0CSU2gJj2ix3yjIrvaI


https://platform.classiq.io/circuit/31oOtYmg0CSU2gJj2ix3yjIrvaI?login=True&version=0.90.0

Ex.1 Results:


,res,count,probability,bitstring
0,"[0, 0, 0]",1031,0.503418,000
1,"[1, 1, 1]",1017,0.496582,111


### Solution 2: GHZ with Quantum Numbers


In [19]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 0]]):
    # Create GHZ state with QNum (gives 0 and -1)
    allocate(x)
    create_ghz_state(x)


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.2 Results:")
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31oOtvlDpg1wzlcFyxTlQYneZsT


https://platform.classiq.io/circuit/31oOtvlDpg1wzlcFyxTlQYneZsT?login=True&version=0.90.0

Ex.2 Results:


,x,count,probability,bitstring
0,-1,1053,0.51416,111
1,0,995,0.48584,000


### Solution 3: Arithmetic Expressions with Automatic Type Inference

#### Part A: Numeric type inference

In [20]:
from classiq import *


@qfunc
def main(a: Output[QNum[2]], b: Output[QNum[2]], c: Output[QNum]):
    allocate(a)
    allocate(b)
    # Put a and b in equal superposition
    hadamard_transform(a)
    hadamard_transform(b)

    # Assign the value of 3*a + b to c
    c |= 3 * a + b

    # Print out c's inferred size in qubits
    print(f"The size of c is {c.size}")


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.4A Results:")
    display(res.dataframe)

The size of c is 4
Quantum program link: https://platform.classiq.io/circuit/31oOuSva3La4Eec6VwsnrShUsEq


https://platform.classiq.io/circuit/31oOuSva3La4Eec6VwsnrShUsEq?login=True&version=0.90.0

Ex.4A Results:


,a,b,c,count,probability,bitstring
0,3,3,12,149,0.072754,11001111
1,3,1,10,140,0.068359,10100111
2,0,2,2,139,0.067871,00101000
3,2,1,7,138,0.067383,01110110
4,1,0,3,135,0.065918,00110001
5,0,3,3,133,0.064941,00111100
6,0,1,1,129,0.062988,00010100
7,2,2,8,129,0.062988,10001010
8,2,3,9,129,0.062988,10011110
9,2,0,6,128,0.062500,01100010


#### Part B: Numeric type inference with fixed-point fractions

In [21]:
# Part B: Fractioned quantum numbers (explicit syntax)
from classiq import *


@qfunc
def main(
    a: Output[QNum[2, UNSIGNED, 1]], b: Output[QNum[2, UNSIGNED, 2]], c: Output[QNum]
):
    # Allocate a and b, then put in superposition
    allocate(a)
    allocate(b)
    hadamard_transform(a)
    hadamard_transform(b)

    # Assign the value of 3*a + b to c
    c |= 3 * a + b

    # Print out the numeric attributes of the inferred type
    print("Numeric attributes of c:")
    print(
        f"size: {c.size}, is_signed: {c.is_signed}, fraction_digits: {c.fraction_digits}"
    )


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.4B Results:")
    display(res.dataframe)

Numeric attributes of c:
size: 5, is_signed: False, fraction_digits: 2
Quantum program link: https://platform.classiq.io/circuit/31oOurORrD3BjMH0lY2OKB4irYP


https://platform.classiq.io/circuit/31oOurORrD3BjMH0lY2OKB4irYP?login=True&version=0.90.0

Ex.4B Results:


,a,b,c,count,probability,bitstring
0,0.0,0.50,0.50,152,0.074219,000101000
1,0.5,0.00,1.50,147,0.071777,001100001
2,1.0,0.75,3.75,140,0.068359,011111110
3,0.0,0.25,0.25,135,0.065918,000010100
4,1.0,0.50,3.50,134,0.065430,011101010
5,1.0,0.00,3.00,131,0.063965,011000010
6,0.5,0.50,2.00,129,0.062988,010001001
7,0.0,0.00,0.00,127,0.062012,000000000
8,1.5,0.25,4.75,127,0.062012,100110111
9,1.0,0.25,3.25,126,0.061523,011010110


### Solution 4: Conditional Operations


In [22]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 2]], flag: Output[QBit]):
    allocate(x)
    hadamard_transform(x)

    allocate(flag)
    # Flip the state of flag if x < 0.5
    control(x < 0.5, lambda: X(flag))  # Flip the state of flag if x < 0.5


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.3 Results:")
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31oOvG1Sigdbup1veciDPF6JMFa


https://platform.classiq.io/circuit/31oOvG1Sigdbup1veciDPF6JMFa?login=True&version=0.90.0

Ex.3 Results:


,x,flag,count,probability,bitstring
0,-0.75,1,295,0.144043,1101
1,0.75,0,275,0.134277,0011
2,-1.00,1,264,0.128906,1100
3,-0.25,1,253,0.123535,1111
4,0.25,1,252,0.123047,1001
5,-0.50,1,246,0.120117,1110
6,0.00,1,233,0.113770,1000
7,0.50,0,230,0.112305,0010


### Solution 5: Computing x² Using Phase Operations

#### Part A: Phase Encoding x²

In [23]:
import numpy as np

from classiq import *
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences


@qfunc
def main(x: Output[QNum[3]]):
    allocate(x)
    # Put x in uniform superposition
    hadamard_transform(x)

    # Encode x² into the phase
    phase(x**2, 2 * np.pi / 8)


# Synthesize the model and show
qprog = synthesize(main)
# show(qprog)

# Specify execution preferences for state vector simulation
preferences = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(backend_name="simulator_statevector"),
)

# Execute and print results:
with ExecutionSession(qprog, preferences) as es:
    res = es.sample()
    print("Ex.6A Results:")
    display(res.dataframe)

Ex.6A Results:


,x,amplitude,probability,bitstring
0,4,0.353553-0.000000j,0.125,100
1,0,0.353553+0.000000j,0.125,000
2,5,0.250000+0.250000j,0.125,101
3,1,0.250000+0.250000j,0.125,001
4,7,0.250000+0.250000j,0.125,111
5,3,0.250000+0.250000j,0.125,011
6,2,-0.353553+0.000000j,0.125,010
7,6,-0.353553+0.000000j,0.125,110


#### Bonus: Fourier Arithmetic


In [24]:
import numpy as np

from classiq import *


@qfunc
def main(x: Output[QNum[3]], res: Output[QNum[4]]):
    allocate(x)
    hadamard_transform(x)
    allocate(res)

    # Use within_apply with QFT and phase operations
    within_apply(
        lambda: qft(res),
        lambda: phase(
            res * (x**2), 2 * np.pi / (2**res.size)
        ),  # evaluates res += x**2  in the Fourier basis
    )


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
# show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.6B Results:")
    res.dataframe.sort_values(by="x", inplace=True)  # Sort by x for better readability
    display(res.dataframe)

Ex.6B Results:


,x,res,count,probability,bitstring
7,0,0,245,0.119629,0000000
4,1,1,253,0.123535,0001001
5,2,4,253,0.123535,0100010
0,3,9,273,0.133301,1001011
3,4,0,256,0.125000,0000100
2,5,9,258,0.125977,1001101
6,6,4,250,0.122070,0100110
1,7,1,260,0.126953,0001111
